In [1]:
#Import all the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
%matplotlib inline

In [2]:
#Import the training and test datasets
titanic = pd.read_csv('C:/Users/927278/Documents/Titanic Kaggle/train.csv')
t_test = pd.read_csv('C:/Users/927278/Documents/Titanic Kaggle/test.csv')

In [60]:
#Check the data and if any fields have missing values
#titanic['Ticket'].map( lambda tick: tick.split(' ')[0][0])
#titanic.filter(items=['Ticket'],like='A*')
#list1 = filter( lambda x: x[0].isalpha(),titanic['Ticket'])
#titanic[titanic['Ticket'][0:]=='A']
#titanic.isnull().sum()

TypeError: Keyword arguments `items`, `like`, or `regex` are mutually exclusive

In [4]:
js = pd.get_dummies(titanic['Sex'],drop_first=True)
js2 = pd.get_dummies(titanic['Embarked'],drop_first=True)

In [16]:
#Create a new variable from passenger titles
title = pd.DataFrame()
title['Title'] = titanic['Name'].map( lambda name: name.split( ',' )[1].split( '.' )[0].strip() )

# map all the titles
Title_Dictionary = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"
                    }

title['Title'] = title.Title.map(Title_Dictionary)
title = pd.get_dummies(title.Title)

In [17]:
titanic_new = titanic.drop(['Sex','Name','Ticket','Cabin','SibSp','Embarked'],axis =1 )
titanic2 = pd.concat([titanic_new,js,js2,title],axis=1)
#Impute
titanic2['Age'].fillna(titanic2['Age'].mean(),inplace=True)
titanic2['Fare'].fillna(titanic2['Fare'].mean(),inplace=True)

In [18]:
#Select X and y
X = titanic2.drop(['PassengerId','Survived'],axis=1)
y = titanic2['Survived']

In [19]:
#Perform Logistic regression
logmodel = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[143  19]
 [ 30  76]]
             precision    recall  f1-score   support

          0       0.83      0.88      0.85       162
          1       0.80      0.72      0.76       106

avg / total       0.82      0.82      0.82       268



In [23]:
logmodel.score(X_test,y_test)

0.8171641791044776